In [ ]:
import torch
import json
import torch.nn.functional as F
# from self_attn_sumnet import FullAttenSUMNet, SUMNet
# from unet import UNet
from sasa_einsum_sumnet import FullAttenSUMNet
from train import test
from tqdm import tqdm
import torchmetrics.functional as TF
from dataloader import LiverDataLoader
from torch.utils import data

In [ ]:
import torchvision
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

In [ ]:
# model_demo = FullAttenSUMNet(in_ch=1, out_ch=2)
# print(model_demo.state_dict()['conv1.rel_h'][10])

In [ ]:
model = FullAttenSUMNet(in_ch=1, out_ch=2)
model_path = '/home/rakshith/miccai_2022/model_weights/segmentation/ssa_seg_adam_1e4_e15_re/'
checkpoint= torch.load(model_path+'model_best_dice.pt')
# checkpoint= torch.load(model_path+'model_last.pt')
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
# model_name = 'ssa_seg_adam_1e4_e15_re'
# config = {
#         "savepath":"/home/rakshith/miccai_2022/model_weights/segmentation/"+model_name+"/",
#         "datapath": "/home/rakshith/Datasets/Task03_Liver/np_array_slices/",
#         "data_split": "/home/rakshith/miccai_2022/segmentation/train_valid_split.json",
#         "out_ch" : 2,
#         "model_name":model_name,
#         "model" : 'ssa',
#         "quantised": 1
#     }
    
# data_path = config['datapath']
# load_path = config['savepath']
# data_split = config['data_split']
# with open(data_split) as f3:
#         data_split = json.load(f3)
# file_list = data_split['valid'][:5]

# # train_split = data_split['train'][:5]
# # train_d_set = LiverDataLoader(datapath=data_path, file_list=train_split, is_transform=True, augmentation=None)

# # train_data_loader = data.DataLoader(
#                                 # train_d_set, batch_size=1,
#                                 # shuffle=True, num_workers=8, pin_memory=True,drop_last=True)

# testDset = LiverDataLoader(datapath=data_path, file_list=file_list,is_transform=True)

# testDataLoader = data.DataLoader(
#                             testDset, batch_size=1, drop_last=True,
#                             shuffle=False, num_workers=2, pin_memory=True)
# model.cuda().eval()

# for datasample in testDataLoader:
#     inputs, mask, fname = datasample
#     print(fname)
    
#     inputs = inputs.cuda()
#     mask = mask.cuda()
    
#     with torch.no_grad():
#         seg_out = model(inputs)
#         net_out_sf = F.softmax(seg_out.data, dim=1)

#         preds = torch.argmax(net_out_sf, dim=1)
#         test_dice = TF.f1_score(preds.flatten().long(), mask[:,1].flatten().long(), num_classes=2, ignore_index=0)
#         print(test_dice)

In [ ]:
# import matplotlib.pyplot as plt
# plt.subplot(121)
# plt.imshow(mask[:,1].squeeze(0).cpu(),cmap='gray')
# plt.subplot(122)
# plt.imshow(preds.squeeze(0).detach().cpu(),cmap='gray')

In [ ]:
# model_name = 'ssa_seg_adam_1e4_e15_re'
# config = {
#         "savepath":"/home/rakshith/miccai_2022/model_weights/segmentation/"+model_name+"/",
#         "datapath": "/home/rakshith/Datasets/Task03_Liver/np_array_slices/",
#         "data_split": "/home/rakshith/miccai_2022/segmentation/train_valid_split.json",
#         "out_ch" : 2,
#         "model_name":model_name,
#         "model" : 'ssa',
#         "quantised": 1
#     }
# model.eval()
# test(config, model)

In [ ]:
import torch
from torch.quantization import get_default_qconfig
# Note that this is temporary, we'll expose these functions to torch.quantization after official releasee
from torch.quantization.quantize_fx import prepare_fx, convert_fx
import json
from torch.utils import data
from dataloader import LiverDataLoader

model_name = 'ssa_seg_adam_1e4_e15_re'
config = {
        "savepath":"/home/rakshith/miccai_2022/model_weights/segmentation/"+model_name+"/",
        "datapath": "/home/rakshith/Datasets/Task03_Liver/np_array_slices/",
        "data_split": "/home/rakshith/miccai_2022/segmentation/train_valid_split.json",
        "out_ch" : 2,
        "model_name":model_name,
        "model" : 'ssa_quant',
        "quantised": 1
    }

data_split = config['data_split']

with open('/home/rakshith/miccai_2022/segmentation/train_valid_split.json') as f3:
    data_split = json.load(f3)
file_list = data_split['valid'][:60]
data_path = '/home/rakshith/Datasets/Task03_Liver/np_array_slices/'

testDset = LiverDataLoader(datapath=data_path, file_list=file_list,is_transform=True)

testDataLoader = data.DataLoader(
                            testDset, batch_size=1, drop_last=True,
                            shuffle=False, num_workers=2, pin_memory=True)
model.eval()
qconfig = get_default_qconfig("fbgemm")
qconfig_dict = {"": qconfig}
prepare_custom_config_dict = {
    # option 1
    "non_traceable_module_name": ["pool"],
}
def calibrate(model, data_loader):
    model.eval()
    with torch.no_grad():
        for image, target,_ in tqdm(data_loader):
            model(image)

prepared_model = prepare_fx(model.cpu(), qconfig_dict,prepare_custom_config_dict=prepare_custom_config_dict,)  # fuse modules and insert observers
calibrate(prepared_model, testDataLoader)  # run calibration on sample data
quantized_model = convert_fx(prepared_model)  # convert the calibrated model to a quantized model

In [ ]:
for datasample in testDataLoader:
    inputs, mask, fname = datasample
    print(fname)
    
    # inputs = inputs
    # mask = mask
    
    with torch.no_grad():
        seg_out = quantized_model(inputs)
        net_out_sf = F.softmax(seg_out.data, dim=1)

        preds = torch.argmax(net_out_sf, dim=1)
        test_dice = TF.f1_score(preds.flatten().long(), mask[:,1].flatten().long(), num_classes=2, ignore_index=0)
        print(test_dice)

In [ ]:
backend = "fbgemm"
model.eval()
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)
state = {'state_dict': model_static_quantized.state_dict() }
save_path = '/home/rakshith/miccai_2022/model_weights/segmentation/ssa_seg_adam_1e4_e15_re_quantised/'
if not os.path.exists(save_path):
    os.mkdir(save_path)
torch.save(state, save_path+'model_best_dice.pt')

In [ ]:
model_name = 'ssa_seg_adam_1e4_e15_re_quantised'
config = {
        "savepath":"/home/rakshith/miccai_2022/model_weights/segmentation/"+model_name+"/",
        "datapath": "/home/rakshith/Datasets/Task03_Liver/np_array_slices/",
        "data_split": "/home/rakshith/miccai_2022/segmentation/train_valid_split.json",
        "out_ch" : 2,
        "model_name":model_name,
        "model" : 'ssa_quant',
        "quantised": 1
    }
test(config, model_static_quantized)

In [ ]:

# torch.cuda.set_device(1)
model = SUMNet(in_ch=1,out_ch=2)
model_path = '/home/rakshith/miccai_2022/model_weights/segmentation/sumnet_seg_adam_1e4_e15/'
model.load_state_dict(torch.load(model_path+'model_best_seg.pt'))

In [ ]:
model.eval()
model= model.cuda()
model_name = 'sumnet_seg_adam_1e4_e15_re_quantised'
config = {
        "savepath":"/home/rakshith/miccai_2022/model_weights/segmentation/"+model_name+"/",
        "datapath": "/home/rakshith/Datasets/Task03_Liver/np_array_slices/",
        "data_split": "/home/rakshith/miccai_2022/segmentation/train_valid_split.json",
        "out_ch" : 2,
        "model_name":model_name,
        "model" : 'sumnet_quant',
        "quantised": 1
    }
test(config, model)

In [ ]:
import torch
from torch.quantization import get_default_qconfig
# Note that this is temporary, we'll expose these functions to torch.quantization after official releasee
from torch.quantization.quantize_fx import prepare_fx, convert_fx
import json
from torch.utils import data
from dataloader import LiverDataLoader

data_split = config['data_split']

with open('/home/rakshith/miccai_2022/segmentation/train_valid_split.json') as f3:
    data_split = json.load(f3)
file_list = data_split['valid'][:10]
data_path = '/home/rakshith/Datasets/Task03_Liver/np_array_slices/'

testDset = LiverDataLoader(datapath=data_path, file_list=file_list,is_transform=True)

testDataLoader = data.DataLoader(
                            testDset, batch_size=1, drop_last=True,
                            shuffle=False, num_workers=2, pin_memory=True)
model.eval()


qconfig = get_default_qconfig("fbgemm")
qconfig_dict = {"": qconfig}
prepare_custom_config_dict = {
    # option 1
    "non_traceable_module_name": ["pool"],
}
def calibrate(model, data_loader):
    model.eval()
    with torch.no_grad():
        for image, target,_ in data_loader:
            model(image)

prepared_model = prepare_fx(model, qconfig_dict,prepare_custom_config_dict=prepare_custom_config_dict,)  # fuse modules and insert observers
calibrate(prepared_model, testDataLoader)  # run calibration on sample data
quantized_model = convert_fx(prepared_model)  # convert the calibrated model to a quantized model

In [ ]:
torch.save(quantized_model.state_dict(),'sumnet_quantized.pt')

In [ ]:
test(config, quantized_model)

In [ ]:
backend = "fbgemm"
model.eval()
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized_sumnet = torch.quantization.prepare(model, inplace=True)
model_static_quantized_sumnet = torch.quantization.convert(model_static_quantized_sumnet, inplace=True)

UNET

In [ ]:
model = UNet(n_channels=1,n_classes=2)
model_path = '/home/rakshith/miccai_2022/model_weights/segmentation/unet_seg_adam_1e4_e15/'
model.load_state_dict(torch.load(model_path+'model_best_seg.pt'))

In [ ]:
backend = "qnnpack"
model.eval()
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized_unet = torch.quantization.prepare(model, inplace=True)
model_static_quantized_unet = torch.quantization.convert(model_static_quantized_unet, inplace=True)

In [ ]:
import torch
from torch.quantization import get_default_qconfig
# Note that this is temporary, we'll expose these functions to torch.quantization after official releasee
from torch.quantization.quantize_fx import prepare_fx, convert_fx
import json
from torch.utils import data
from dataloader import LiverDataLoader

model_name = 'unet_seg_adam_1e4_e15_re_quantised'
config = {
        "savepath":"/home/rakshith/miccai_2022/model_weights/segmentation/"+model_name+"/",
        "datapath": "/home/rakshith/Datasets/Task03_Liver/np_array_slices/",
        "data_split": "/home/rakshith/miccai_2022/segmentation/train_valid_split.json",
        "out_ch" : 2,
        "model_name":model_name,
        "model" : 'unet_quant',
        "quantised": 1
    }

data_split = config['data_split']

with open('/home/rakshith/miccai_2022/segmentation/train_valid_split.json') as f3:
    data_split = json.load(f3)
file_list = data_split['valid'][:10]
data_path = '/home/rakshith/Datasets/Task03_Liver/np_array_slices/'

testDset = LiverDataLoader(datapath=data_path, file_list=file_list,is_transform=True)

testDataLoader = data.DataLoader(
                            testDset, batch_size=1, drop_last=True,
                            shuffle=False, num_workers=2, pin_memory=True)
model.eval()
qconfig = get_default_qconfig("qnnpack")
qconfig_dict = {"": qconfig}
prepare_custom_config_dict = {
    # option 1
    "non_traceable_module_name": ["up1","up2","up3","up4","ConvTranspose2d"],
}
def calibrate(model, data_loader):
    model.eval()
    with torch.no_grad():
        for image, target,_ in data_loader:
            model(image)

prepared_model = prepare_fx(model, qconfig_dict,prepare_custom_config_dict=prepare_custom_config_dict,)  # fuse modules and insert observers
calibrate(prepared_model, testDataLoader)  # run calibration on sample data
quantized_model_unet = convert_fx(prepared_model)  # convert the calibrated model to a quantized model

In [ ]:

test(config, quantized_model_unet)

In [ ]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), config['savepath']+"resnet50_quant.pth")
    print("%.2f MB" %(os.path.getsize(config['savepath']+"resnet50_quant.pth")/1e6))


In [ ]:
print_model_size(model_static_quantized)
print_model_size(model_static_quantized_unet)
print_model_size(model_static_quantized_sumnet)